You can create a simple interactive GUI in Python using Streamlit, a lightweight library that's perfect for deploying applications quickly, including on Google Colab. Streamlit will allow you to integrate text, images, and interact with GPT-4 API, all from a single interface with a friendly user experience.

Here’s how you can set up this interactive folklore exploration tool in Streamlit. The following code includes a GUI where users can explore the dilemmas, see paired stories, and interact with a GPT-4 API for deeper analysis or explanation of each tale. We’ll also add options for adaptive user control and seamless integration with GitHub for version tracking.

###Steps to Create and Deploy the Demo App
* Install Streamlit and other required libraries.
* Build the Streamlit GUI with options for choosing dilemmas and displaying the story pairs.
* Integrate GPT-4 API for analysis and interpretation.
* Deploy the app on Google Colab and save it to GitHub

#1. Install Streamlit and Required Libraries
In Google Colab, install Streamlit, requests (for API calls), and ngrok (for hosting the Streamlit app).

In [ ]:
!pip install streamlit requests pyngrok

###2. **Set Up the Ngrok Authentication Token**
Replace "your_authtoken" with your actual ngrok token in the following cell.

In [ ]:
#!ngrok config add-authtoken "your_authtoken"
#!ngrok authtoken "your_authtoken"
!ngrok config add-authtoken "2ol9K0GNdjTzB2GrP2i2LES3WVu_EDz1dLjEE5YJ2yMjXAob"
!ngrok authtoken "2ol9K0GNdjTzB2GrP2i2LES3WVu_EDz1dLjEE5YJ2yMjXAob"


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


###3. **Mount Google drive for Excel Databases**
Mount to /content/drive

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

###4. **Save Your Streamlit App Code in app.py**
In a new Colab cell, copy your Streamlit app code and save it as app.py:

In [ ]:
code = """
# Import required libraries
import pandas as pd
import streamlit as st
import matplotlib.pyplot as plt
import numpy as np

# Define styling rules based on POS tags
styling_rules = {
    "NOUN": {"color": "blue", "bold": True},
    "PRON": {"color": "purple", "italic": True},
    "VERB": {"color": "green", "underline": True},
    "ADV": {"color": "orange"},
    "CONJ": {"color": "red", "bold": True},
    "ADP": {"color": "cyan"},
    "DET": {"color": "gray", "italic": True},
    "PUNCT": {"color": "black"},
    "PROPN": {"color": "darkblue", "bold": True, "italic": True},
    "INTJ": {"color": "pink", "italic": True},
    "NUM": {"color": "teal"},
    "PART": {"color": "brown"},
    "SCONJ": {"color": "darkred"},
    "CCONJ": {"color": "lightcoral"},
    "ADJ": {"color": "magenta", "italic": True},
    "TEXT": {"color": "brown", "font_name": "Courier New"}
}

def generate_filtered_pos_key(pos_tags_used, sentence_pos_tags):

    #Creates a compact markdown table showing only the POS that appear in the current sentence,
    #with examples derived from the sentence's POS tags.
    #:param pos_tags_used: Set of POS tags found in the current sentence.
    #:param sentence_pos_tags: Full list of <word|POS> pairs from the current sentence.
    #:return: Filtered POS Key as a markdown table.

    key_markdown = "<h5>Parts of Speech Key</h5>"
    key_markdown += "<table style='border-collapse: collapse; width: 100%; font-size: small;'>"
    key_markdown += "<tr><th style='text-align: left;'>POS</th><th style='text-align: left;'>Example</th></tr>"

    # Build a dictionary of POS -> example word from sentence
    pos_to_example = {}
    for tag in sentence_pos_tags:
        if "|" in tag and tag.startswith("<") and tag.endswith(">"):
            word, pos = tag.strip("<>").rsplit('|', 1)
            if pos not in pos_to_example:  # Only take the first occurrence
                pos_to_example[pos] = word

    for pos, style in styling_rules.items():
        if pos in pos_tags_used:  # Only include POS tags that are present
            pos_label = {
                "NOUN": "Noun",
                "PRON": "Pronoun",
                "VERB": "Verb",
                "ADV": "Adverb",
                "CONJ": "Conjunction",
                "ADP": "Preposition",
                "DET": "Determiner",
                "PUNCT": "Punctuation",
                "PROPN": "Proper Noun",
                "INTJ": "Interjection",
                "NUM": "Number",
                "PART": "Particle",
                "SCONJ": "Subordinating Conjunction",
                "CCONJ": "Coordinating Conjunction",
                "ADJ": "Adjective",
                "TEXT": "Direct Speech",
            }.get(pos, pos)

            # Example word for the part of speech
            example_word = pos_to_example.get(pos, "N/A")
            color = f"color:{style.get('color', 'black')};"
            bold = "font-weight:bold;" if style.get("bold", False) else ""
            italic = "font-style:italic;" if style.get("italic", False) else ""
            underline = "text-decoration:underline;" if style.get("underline", False) else ""
            font_name = f"font-family:{style.get('font_name', 'Arial')};"

            # Styled example
            styled_example = f'<span style="{color} {bold} {italic} {underline} {font_name}">{example_word}</span>'

            # Add row to the table
            key_markdown += f"<tr><td>{pos_label}</td><td>{styled_example}</td></tr>"

    key_markdown += "</table>"
    return key_markdown


# Function to generate a colorized key for parts of speech
def generate_pos_key():

    # Creates a compact markdown table showing the POS key with corresponding styles, reduced font size,
    # and no spacing between rows.

    key_markdown = "<h5>Parts of Speech Key</h5>"
    key_markdown += "<table style='border-collapse: collapse; width: 100%; font-size: small;'>"
    key_markdown += "<tr><th style='text-align: left;'>POS</th><th style='text-align: left;'>Example</th></tr>"

    for pos, style in styling_rules.items():
        # Label for the part of speech
        pos_label = {
            "NOUN": "Noun",
            "PRON": "Pronoun",
            "VERB": "Verb",
            "ADV": "Adverb",
            "CONJ": "Conjunction",
            "ADP": "Preposition",
            "DET": "Determiner",
            "PUNCT": "Punctuation",
            "PROPN": "Proper Noun",
            "INTJ": "Interjection",
            "NUM": "Number",
            "PART": "Particle",
            "SCONJ": "Subordinating Conjunction",
            "CCONJ": "Coordinating Conjunction",
            "ADJ": "Adjective",
            "TEXT": "Direct Speech",
        }.get(pos, pos)

        # Generate styled example for the part of speech
        example_word = f"{pos_label}"  # Use label as the example
        color = f"color:{style.get('color', 'black')};"
        bold = "font-weight:bold;" if style.get("bold", False) else ""
        italic = "font-style:italic;" if style.get("italic", False) else ""
        underline = "text-decoration:underline;" if style.get("underline", False) else ""
        font_name = f"font-family:{style.get('font_name', 'Arial')};"
        styled_example = f'<span style="{color} {bold} {italic} {underline} {font_name}">{example_word}</span>'

        # Add row to the table
        key_markdown += f"<tr style='margin: 0; padding: 0;'><td style='padding: 0;'>{pos_label}</td><td style='padding: 0;'>{styled_example}</td></tr>"

    key_markdown += "</table>"
    return key_markdown

# Function to style a sentence using POS tags# Function to style a sentence using POS tags

def style_sentence_with_pos(sentence, pos_tags):

    #Styles a sentence using POS tags and styling rules.
    #:param sentence: Original sentence as a string.
    #:param pos_tags: POS tags as a list of strings in the format <word|POS>.
    #:return: Styled HTML string.

    styled_words = []
    for word_pos in pos_tags:
        try:
            # Check for proper formatting and split word and POS
            if "|" in word_pos and word_pos.startswith("<") and word_pos.endswith(">"):
                word, pos = word_pos.strip('<>').rsplit('|', 1)
            else:
                # If no POS tag, treat as punctuation or unknown
                word = word_pos.strip('<>')
                pos = "PUNCT" if word in [".", ",", "!", "?", ";", ":"] else "UNKNOWN"

            # Remove any residual "ket" format from the word
            if "|" in word:
                word = word.split('|')[0]  # Keep only the actual word part

            # Get style based on POS
            style = styling_rules.get(pos, {"color": "black"})  # Default to black
            color = f"color:{style.get('color', 'black')};"
            bold = "font-weight:bold;" if style.get("bold", False) else ""
            italic = "font-style:italic;" if style.get("italic", False) else ""
            underline = "text-decoration:underline;" if style.get("underline", False) else ""
            font_name = f"font-family:{style.get('font_name', 'Arial')};"

            # Avoid styling punctuation
            if pos == "PUNCT":
                styled_word = word  # Render punctuation as is
            else:
                styled_word = f'<span style="{color} {bold} {italic} {underline} {font_name}">{word}</span>'
            styled_words.append(styled_word)
        except Exception as e:
            # Handle malformed tags gracefully
            styled_words.append(f'<span style="color:red;">{word_pos}</span>')  # Highlight problematic tags

    return ' '.join(styled_words)
#
def generate_filtered_pos_key(pos_tags_used, sentence_pos_tags):

    #Creates a compact markdown table showing only the POS that appear in the current sentence,
    #with examples derived from the sentence's POS tags.
    #:param pos_tags_used: Set of POS tags found in the current sentence.
    #:param sentence_pos_tags: Full list of <word|POS> pairs from the current sentence.
    #S:return: Filtered POS Key as a markdown table.

    key_markdown = "<h5>Parts of Speech Key</h5>"
    key_markdown += "<table style='border-collapse: collapse; width: 100%; font-size: small;'>"
    key_markdown += "<tr><th style='text-align: left;'>POS</th><th style='text-align: left;'>Example</th></tr>"

    # Build a dictionary of POS -> example word from sentence
    pos_to_example = {}
    for tag in sentence_pos_tags:
        if "|" in tag and tag.startswith("<") and tag.endswith(">"):
            word, pos = tag.strip("<>").rsplit('|', 1)
            if pos not in pos_to_example:  # Only take the first occurrence
                pos_to_example[pos] = word

    for pos, style in styling_rules.items():
        if pos in pos_tags_used:  # Only include POS tags that are present
            pos_label = {
                "NOUN": "Noun",
                "PRON": "Pronoun",
                "VERB": "Verb",
                "ADV": "Adverb",
                "CONJ": "Conjunction",
                "ADP": "Preposition",
                "DET": "Determiner",
                "PUNCT": "Punctuation",
                "PROPN": "Proper Noun",
                "INTJ": "Interjection",
                "NUM": "Number",
                "PART": "Particle",
                "SCONJ": "Subordinating Conjunction",
                "CCONJ": "Coordinating Conjunction",
                "ADJ": "Adjective",
                "TEXT": "Direct Speech",
            }.get(pos, pos)

            # Example word for the part of speech
            example_word = pos_to_example.get(pos, "N/A")
            color = f"color:{style.get('color', 'black')};"
            bold = "font-weight:bold;" if style.get("bold", False) else ""
            italic = "font-style:italic;" if style.get("italic", False) else ""
            underline = "text-decoration:underline;" if style.get("underline", False) else ""
            font_name = f"font-family:{style.get('font_name', 'Arial')};"

            # Styled example
            styled_example = f'<span style="{color} {bold} {italic} {underline} {font_name}">{example_word}</span>'

            # Add row to the table
            key_markdown += f"<tr style='padding: 0; margin: 0;'><td style='padding: 0;'>{pos_label}</td><td style='padding: 0;'>{styled_example}</td></tr>"
    key_markdown += "</table>"
    return key_markdown

# Streamlit app setup
st.title("HOW KWAKU ANANSE (THE SPIDER) GOT ASO IN MARRIAGE")

# Load Story and Comprehensive Campbell Dataset
story_url = "https://raw.githubusercontent.com/BillWorstell/Akan/main/Ananse1/Ananse1_Learned.xlsx"

import requests

# URL of the Markdown file on GitHub
md_file_url = "https://raw.githubusercontent.com/BillWorstell/Akan/main/Ananse1/DramaticAnalysis.md"

try:
    # Fetch the content of the file
    response = requests.get(md_file_url)
    response.raise_for_status()  # Raise an error if the request failed
    dramatic_analysis_content = response.text
except requests.exceptions.RequestException as e:
    dramatic_analysis_content = f"Error fetching Dramatic Analysis content: {e}"

# Add a toggle button at the top of the sidebar
toggle_option = st.sidebar.radio(
    "Choose Sidebar Content",
    ("Current Page Content", "Dramatic Analysis"),
)
# Load the Excel file
story_file = pd.ExcelFile(story_url, engine="openpyxl")

# Story levels and sentence selector
story_levels = story_file.sheet_names
level_choice = st.sidebar.selectbox("Story Level", story_levels)
story_data = story_file.parse(level_choice)

num_sentences = len(story_data)

# Initialize session state for sentence index
if "sentence_index" not in st.session_state:
  st.session_state.sentence_index = 0

# Sync slider with session state
sentence = st.sidebar.slider(
  "Sentence Number?",
  0, num_sentences - 1,
  st.session_state.sentence_index
)

# Update session state when slider changes
if sentence != st.session_state.sentence_index:
  st.session_state.sentence_index = sentence

# Add navigation buttons in the sidebar
if st.sidebar.button("Previous Line"):
  st.session_state.sentence_index = max(0, st.session_state.sentence_index - 1)
if st.sidebar.button("Next Line"):
  st.session_state.sentence_index = min(num_sentences - 1, st.session_state.sentence_index + 1)

# Sync slider value with navigation buttons
sentence = st.session_state.sentence_index

# Conditionally display content based on the toggle
if toggle_option == "Dramatic Analysis":
    # Display the Dramatic Analysis markdown content
    st.sidebar.markdown("---")  # Separator for clarity
    st.sidebar.markdown("### Dramatic Analysis")
    st.sidebar.markdown(dramatic_analysis_content, unsafe_allow_html=True)
else:



    # Display the POS key in the sidebar
    pos_key = generate_pos_key()

    # Collect POS tags used in the current sentence
    current_pos_tags = set()
    sentence_pos_tags = []  # List of all <word|POS> pairs in the sentence
    for col_idx in [4, 5, 6, 7]:  # POS columns for Akan, English, AtoE, EtoA
      column_data = story_data.iloc[sentence, col_idx]
      if isinstance(column_data, str):  # If data is a string, split it
          column_data = column_data.split()
      elif isinstance(column_data, list):  # If it's already a list, use it
          pass
      else:
          column_data = []  # Default to an empty list if unexpected type
      current_pos_tags.update([tag.rsplit('|', 1)[1].strip("<>") for tag in column_data if "|" in tag])
      sentence_pos_tags.extend(column_data)

    st.sidebar.write(generate_filtered_pos_key(current_pos_tags, sentence_pos_tags), unsafe_allow_html=True)

    # Parse and clean the Act and Scene data
    story_data["Scene Number"] = story_data["Scene"].str.extract(r"Scene (\d+)").astype(int)
    story_data["Cumulative Scene"] = story_data["Scene Number"].cumsum()

    # Create the plot for cumulative scenes
    fig, ax = plt.subplots(figsize=(4, 3))
    x = np.arange(len(story_data))
    y = story_data["Cumulative Scene"]

    ax.plot(x, y, marker="o", label="Cumulative Scene")
    ax.set_xlabel("Line Number")
    ax.set_ylabel("Cumulative Scene Number")
    ax.set_title("Cumulative Scene Progression")

    # Highlight the current sentence
    current_x = sentence
    current_y = story_data.loc[sentence, "Cumulative Scene"]
    current_scene = story_data.loc[sentence, "Scene"]
    ax.scatter([current_x], [current_y], color="red", label=f"Current: {current_scene}")
    ax.annotate(
      f"{current_scene}",
      (current_x, current_y),
      textcoords="offset points",
      xytext=(0, 10),
      ha="center",
      color="red",
    )

    ax.legend()

    # Add the plot to the sidebar
    st.sidebar.pyplot(fig)

#####################################

# Display Act and Scene for current sentence
act = story_data.loc[sentence, "Act"]
scene = story_data.loc[sentence, "Scene"]
st.markdown(f"#### {act}")
st.markdown(f"#### {scene}")

# Main section split into four columns
col1, col2, col3, col4 = st.columns(4)

# Populate Column 1 (Akan)
with col1:
    st.write("### Akan")
    akan_sentence = story_data.iloc[sentence, 0]
    akan_pos_tags = story_data.iloc[sentence, 4].split()
    styled_akan = style_sentence_with_pos(akan_sentence, akan_pos_tags)
    st.markdown(f"#### {styled_akan}", unsafe_allow_html=True)

# Populate Column 2 (AtoE)
with col2:
    st.write("### AtoE")
    atoe_sentence = story_data.iloc[sentence, 2]
    atoe_pos_tags = story_data.iloc[sentence, 6].split()
    styled_atoe = style_sentence_with_pos(atoe_sentence, atoe_pos_tags)
    st.markdown(f"#### {styled_atoe}", unsafe_allow_html=True)

# Populate Column 3 (English)
with col3:
    st.write("### English")
    english_sentence = story_data.iloc[sentence, 1]
    english_pos_tags = story_data.iloc[sentence, 5].split()
    styled_english = style_sentence_with_pos(english_sentence, english_pos_tags)
    st.markdown(f"#### {styled_english}", unsafe_allow_html=True)

# Populate Column 4 (EtoA)
with col4:
    st.write("### EtoA")
    etoa_sentence = story_data.iloc[sentence, 3]
    etoa_pos_tags = story_data.iloc[sentence, 7].split()
    styled_etoa = style_sentence_with_pos(etoa_sentence, etoa_pos_tags)
    st.markdown(f"#### {styled_etoa}", unsafe_allow_html=True)

#st.write("### Dramatic Analysis")
# Display Dramatic Analysis for current sentence
dramatic_analysis = story_data.iloc[sentence, 8]
st.markdown(f"#### {dramatic_analysis}")

"""

with open("app.py", "w") as file:
    file.write(code)

### Key Features of Ananse1StreamlitDrama8.ipynb

1. **Dynamic GUI for Exploring Folklore**:
   - An interactive GUI built using Streamlit allows users to explore and compare folklore stories across multiple languages, including Akan and English.
   - Integrated functionality for displaying Part-of-Speech (POS) tags with visually styled formatting enhances linguistic understanding.

2. **POS-Tagged Sentence Styling**:
   - Sentences are styled dynamically using POS tags with customizable rules for each tag type (e.g., nouns, verbs, adjectives), offering an engaging and educational user experience.

3. **Four-Language Parallel Display**:
   - The app presents parallel translations across four columns: Akan, Akan-to-English (AtoE), English, and English-to-Akan (EtoA).
   - Users can view translations and their POS-tagged versions side-by-side for comparative study.

4. **Customizable Story Navigation**:
   - A user-friendly sidebar allows selection of story levels (e.g., chapters or sections) and specific sentences within each level.
   - Users can explore all parts of the story interactively, selecting levels independently for each language display.

5. **Streamlit and Ngrok Integration for Deployment**:
   - The app is designed to run seamlessly on Google Colab, with automatic deployment to the web via Ngrok for public access.
   - Version control is facilitated by direct integration with GitHub, ensuring reproducibility and collaboration.

6. **POS Label Display**:
   - The app includes a key for Part-of-Speech (POS) labels, allowing users to reference and better understand the tags applied to sentences.

7. **Act and Scene Titles**:
   - Act and Scene titles are displayed prominently at the top of the content area, formatted for readability.

8. **Navigation Widget**:
   - Buttons in the sidebar for "Previous Line" and "Next Line" enable users to easily browse through the story.

9. **Interactive Line Number vs. Scene Plot**:
   - A plot in the sidebar highlights the current line with a distinct red marker and updates dynamically as the user navigates.
   - The plot provides a visual representation of cumulative scenes against line numbers, enhancing contextual understanding.


Key Features:
Act and Scene Titles:

Act and Scene are displayed on separate lines in a smaller font.
Navigation Widget:

Buttons in the sidebar for "Previous Line" and "Next Line".
Line Number vs. Campbell Stage Plot:

A plot highlights the current line with a distinct red marker.

In [ ]:
#!streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py

### 4. **Run the Streamlit App and Create the Ngrok Tunnel**
Now, run the following code to start the Streamlit app and create an ngrok tunnel to access it. Make sure only one instance of Streamlit is running to avoid conflicts.

In [ ]:
import subprocess
# Start Streamlit app on port 8501
subprocess.Popen(["streamlit", "run", "app.py", "--server.port", "8501"])


<Popen: returncode: None args: ['streamlit', 'run', 'app.py', '--server.port...>

In [ ]:
from pyngrok import ngrok

# Start a single ngrok tunnel to port 8501
url = ngrok.connect(8501)
print(f"Access the Streamlit app at: {url}")


Access the Streamlit app at: NgrokTunnel: "https://eeac-34-141-209-176.ngrok-free.app" -> "http://localhost:8501"


###**Step 5: Confirm Streamlit is Running on Port 8501**
To verify that the Streamlit app is accessible, run:

In [ ]:
!curl -I http://localhost:8501


curl: (7) Failed to connect to localhost port 8501 after 0 ms: Connection refused


If Streamlit is running correctly, this command should show a response. If there’s an error, check if the Streamlit app has started correctly in the cell above.

###**Important Notes:**

*   Avoid running multiple cells that initiate ngrok simultaneously, as this will exceed the session limit.
*   If you still face issues, go to the ngrok dashboard to terminate any active sessions before starting a new one in Colab.


.